In [24]:
cd t:\freely_moving_ephys\ephys_recordings\101420\G6H28P6LT\fm1

t:\freely_moving_ephys\ephys_recordings\101420\G6H28P6LT\fm1


In [25]:
#cd \\niell-v2-w7\freely_moving_ephys\ephys_recordings\092820\J524RT\outputs_Oct07a


In [26]:
ls

 Volume in drive T is 8tb internal A
 Volume Serial Number is 7481-8BF5

 Directory of t:\freely_moving_ephys\ephys_recordings\101420\G6H28P6LT\fm1

11/22/2020  11:45 PM    <DIR>          .
11/22/2020  11:45 PM    <DIR>          ..
10/14/2020  01:54 PM    <DIR>          .phy
10/14/2020  11:56 AM     4,091,281,408 101420_G6H28P6LT_fm1_Ephys.bin
10/14/2020  11:56 AM         4,744,512 101420_G6H28P6LT_fm1_Ephys_BonsaiTS.csv
10/14/2020  12:56 PM     2,045,640,704 101420_G6H28P6LT_fm1_Ephys_int16_med1_nch16.bin
11/19/2020  09:39 PM       144,467,879 101420_G6H28P6LT_fm1_Ephys_medianTrace.mat
11/20/2020  09:35 AM         2,501,953 101420_G6H28P6LT_fm1_ephys_merge.json
10/14/2020  11:56 AM       575,188,626 101420_G6H28P6LT_fm1_REYE.avi
10/14/2020  11:56 AM         1,075,571 101420_G6H28P6LT_fm1_REYE_BonsaiTS.csv
11/22/2020  08:18 PM         2,024,560 101420_G6H28P6LT_fm1_REYE_BonsaiTSformatted.csv
11/24/2020  12:35 PM       563,038,508 101420_G6H28P6LT_fm1_REYE_plot.avi
11/24/2020  01:44 PM 

In [27]:
world_file = r'092820_J524RT_fm1world.nc'
eye_file = '092820_J524RT_fm1Reye.nc'
ephys_file = '092820_J524RT_fm1_ephys.json'
top_file = '092820_J524RT_fm1_TOP1.nc'

eye_file =   "101520_G6H28P6LT_fm1Reye.nc"
world_file = '101520_G6H28P6LT_fm1world.nc'
ephys_file = '101520_G6H28P6LT_fm1_ephys_merge.json'
top_file = '101520_G6H28P6LT_fm1_TOP1.nc'

eye_file =   "101420_G6H28P6LT_fm1_Reye.nc"
world_file = '101420_G6H28P6LT_fm1_world.nc'
ephys_file = '101420_G6H28P6LT_fm1_ephys_merge.json'
top_file = '101420_G6H28P6LT_fm1_TOP1.nc'

name_base = ephys_file[0:-5]
name_base

'101420_G6H28P6LT_fm1_ephys_merge'

In [28]:
from netCDF4 import Dataset
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import pickle
import time
from matplotlib.animation import FFMpegWriter
import matplotlib as mpl 
import wavio
mpl.rcParams['animation.ffmpeg_path'] = r'C:\Program Files\ffmpeg\bin\ffmpeg.exe'
from scipy.interpolate import interp1d
from numpy import nan
from scipy.ndimage import shift as imshift

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

In [29]:
top_data = xr.open_dataset(top_file)
top_vid = np.uint8(top_data['TOP1_video'])
top_data

FileNotFoundError: [Errno 2] No such file or directory: b't:\\freely_moving_ephys\\ephys_recordings\\101420\\G6H28P6LT\\fm1\\101420_G6H28P6LT_fm1_TOP1.nc'

In [ ]:
top_vid = np.uint8(top_data['TOP1_video'])
topT = top_data['timestamps']
topT

In [ ]:
# get points from top cam to calculate locomotion
# base of spine seems most reliable
ptNames = top_data['TOP1_pts']
spinex = ptNames.sel(point_loc = 'spine_x').copy()
spiney = ptNames.sel(point_loc = 'spine_y').copy()
spinep = ptNames.sel(point_loc = 'spine_likelihood').copy()

#smooth pts before calculating speed
box = np.ones(31)/31
spinex_sm = np.convolve(spinex, box, mode='same')
spiney_sm = np.convolve(spiney,box,mode = 'same')

#speed
spd = np.sqrt(np.diff(spinex_sm)**2 + np.diff(spiney_sm)**2)*60/10
spd[0:10]=0
plt.plot(spd[0:1800])


In [ ]:
plt.plot(spinex[100:200])
plt.plot(spinex_sm[100:200])

In [ ]:
# load worldcam
world_data = xr.open_dataset(world_file)
world_vid_raw = np.uint8(world_data['WORLD_video'])



In [ ]:
#resize worldcam to make more manageable
sz = world_vid_raw.shape
downsamp = 0.25
world_vid = np.zeros((sz[0],np.int(sz[1]*downsamp),np.int(sz[2]*downsamp)), dtype = 'uint8')
for f in range(sz[0]):
    world_vid[f,:,:] = cv2.resize(world_vid_raw[f,:,:],(np.int(sz[2]*downsamp),np.int(sz[1]*downsamp)))
worldT = world_data.timestamps.copy()
world_data

In [ ]:
# plot worldcam timing
fig, axs = plt.subplots(1,2,figsize=(8,3))
axs[0].plot(np.diff(worldT)); axs[0].set_xlabel('frame'); axs[0].set_ylabel('deltaT'); axs[0].set_title('world cam')
axs[1].hist(np.diff(worldT),100);axs[1].set_xlabel('deltaT')

In [ ]:
# plot mean world image
plt.imshow(np.mean(world_vid,axis=0),vmin=0,vmax=255); plt.title('mean worldcam')

In [ ]:
# read ephys data
ephys_data = pd.read_json(ephys_file)
ephys_data


In [ ]:
# get intitial ephys timepoint (t0)
ephysT0 = ephys_data.iloc[0,12]
ephysT0 

In [ ]:
# select good cells from phy2
goodcells = ephys_data.loc[ephys_data['group']=='good']
goodcells.shape
units = goodcells.index.values
units

In [ ]:
#spike rasters
fig, ax = plt.subplots(figsize=(20,8))
ax.fontsize = 20
for i,ind in enumerate(goodcells.index):
   plt.vlines(goodcells.at[ind,'spikeT'],i-0.25,i+0.25)
   plt.xlim(0, 10); plt.xlabel('secs',fontsize = 20); plt.ylabel('unit #',fontsize=20)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.savefig('spikeraster.pdf')



In [ ]:
# generate wave file
spk =np.array(ephys_data.at[units[1],'spikeT'])

datarate = 30000
rate = 44100    # samples per second
Tmax = 10           # sample duration (seconds)
f = 440.0       # sound frequency (Hz)
# Compute waveform samples
t = np.linspace(0, Tmax, Tmax*datarate,endpoint=False)
x = np.zeros(np.size(t))
for spt in spk[spk<Tmax]:
    x[np.int64(spt*datarate) : np.int64(spt*datarate +30)] = 1
    x[np.int64(spt*datarate)+31 : np.int64(spt*datarate +60)] =- 1
plt.plot(x)
    

# Write the samples to a file
wavio.write("spike.wav", x, datarate, sampwidth=1)


In [ ]:
#load eye data
eye_data = xr.open_dataset(eye_file)
eye_vid = np.uint8(eye_data['REYE_video'])
plt.imshow(eye_vid[0,:,:]); plt.title('eye cam')
eyeT = eye_data.timestamps.copy()
eyeT

In [ ]:
# plot eye timestamps
fig, axs = plt.subplots(1,2,figsize=(8,3))
axs[0].plot(np.diff(eyeT)); axs[0].set_xlabel('frame'); axs[0].set_ylabel('deltaT'); axs[0].set_title('eye cam')
axs[1].hist(np.diff(eyeT),100);

In [ ]:
#plot eye postion across recording
eye_params = eye_data['REYE_ellipse_params']
plt.plot(eye_params.sel(ellipse_params = 'theta')*180/3.1415,eye_params.sel(ellipse_params = 'phi')*180/3.1415,'.')
plt.xlabel('theta'); plt.ylabel('phi')

In [ ]:
# adjust eye/world/top times relative to ephys
eyeT = eye_data.timestamps  - ephysT0
if eyeT[0]<-600:
    eyeT = eyeT + 8*60*60 # 8hr offset for some data
worldT = world_data.timestamps - ephysT0
if worldT[0]<-600:
    worldT = worldT + 8*60*60
topT = top_data.timestamps - ephysT0
eyeT, worldT,topT

In [ ]:
#plot eye variables
fig,axs = plt.subplots(4,1,figsize = (4,8))
for i,val in enumerate(eye_params.ellipse_params[0:4]):
    axs[i].plot(eyeT,eye_params.sel(ellipse_params = val))
    axs[i].set_ylabel(val.values)
    

In [ ]:
# calculate eye veloctiy
dEye = np.diff(eye_params.sel(ellipse_params = 'theta'))*180/3.14159
eyeR = eye_params.sel(ellipse_params = 'longaxis').copy()
eye_vid.shape

In [ ]:
# plot data figure

fig = plt.figure(figsize = (6,8))
gs = fig.add_gridspec(6,2)
axEye = fig.add_subplot(gs[0:2,0])
axWorld = fig.add_subplot(gs[0:2,1])
axTheta = fig.add_subplot(gs[2,:])
axdTheta = fig.add_subplot(gs[3,:])
axR = fig.add_subplot(gs[4:6,:])
#axRad = fig.add_subplot(gs[3,:])

tr = [0,30] # 340-385
fr = np.mean(tr) # time for frame
eyeFr = np.abs(eyeT-fr).argmin(dim = "frame")
worldFr = np.abs(worldT-fr).argmin(dim = "frame")

axEye.cla(); axEye.axis('off'); 
axEye.imshow(eye_vid[eyeFr,:,:],'gray',vmin=0,vmax=255,aspect = "equal")
#axEye.plot(eye_params.sel(ellipse_params = 'X0')[fr]/2,eye_params.sel(ellipse_params = 'Y0')[fr]/2,'r.')
axEye.set_xlim(40,200); axEye.set_ylim(0,120)

axWorld.cla();  axWorld.axis('off'); 
axWorld.imshow(world_vid[worldFr,:,:],'gray',vmin=0,vmax=255,aspect = "equal")
   
#plot eye position
axTheta.cla()
axTheta.plot(eyeT,0.5*eye_params.sel(ellipse_params = 'theta')*180/3.14159)
axTheta.set_xlim(tr[0],tr[1]); 
axTheta.set_ylabel('theta - deg'); axTheta.set_ylim(-30,30)

# plot eye velocity
axdTheta.cla()
axdTheta.plot(eyeT[0:-1],dEye*60/2); ax.set_ylabel('dtheta')
#sacc = np.transpose(np.where(np.abs(dEye)>10))
#axdTheta.plot(sacc,np.sign(dEye[sacc])*20,'.')
axdTheta.set_xlim(tr[0],tr[1]); 
axdTheta.set_ylim(-10*60,10*60); axdTheta.set_ylabel('eye vel - deg/sec')

#axRad.cla()
#axRad.plot(eye_params.sel(ellipse_params = 'longaxis')[frameRange])
#axRad.set_xlim(0,frameRange[-1]-frameRange[0]); 
#axRad.set_ylabel('radius'); axRad.set_xlabel('frame #'); axRad.set_ylim(0,40)

# plot spikes
axR.fontsize = 20
for i,ind in enumerate(goodcells.index):
    axR.vlines(goodcells.at[ind,'spikeT'],i-0.25,i+0.25,'k',linewidth=0.25)
axR.set_xlim(tr[0],tr[1]); axR.set_xlabel('secs'); axR.set_ylabel('unit #')
axR.spines['right'].set_visible(False)
axR.spines['top'].set_visible(False)

plt.tight_layout()

plt.savefig('cams_eyes_spikes_092820_wn_0_30.pdf')

In [ ]:
# normalize world movie and calculate contrast
img_norm = (world_vid-np.mean(world_vid,axis=0))/np.std(world_vid,axis=0)
contrast = np.empty(worldT.size)
for i in range(worldT.size):
    contrast[i] = np.std(img_norm[i,:,:])
plt.plot(worldT[0:1000],contrast[0:1000])
plt.xlabel('time')
plt.ylabel('contrast')

In [ ]:
# plot another data figure (with contrast or speed)

fig = plt.figure(figsize = (6,8))
gs = fig.add_gridspec(7,3)
axEye = fig.add_subplot(gs[0:2,1])
axWorld = fig.add_subplot(gs[0:2,0])
axTop = fig.add_subplot(gs[0:2,2])
axTheta = fig.add_subplot(gs[3,:])
axdTheta = fig.add_subplot(gs[4,:])
axVid = fig.add_subplot(gs[2,:])
axR = fig.add_subplot(gs[5:7,:])
#axRad = fig.add_subplot(gs[3,:])

tr = [0,30] # 340-385
fr = np.mean(tr) # time for frame
eyeFr = np.abs(eyeT-fr).argmin(dim = "frame")
worldFr = np.abs(worldT-fr).argmin(dim = "frame")

axEye.cla(); axEye.axis('off'); 
axEye.imshow(eye_vid[eyeFr,:,:],'gray',vmin=0,vmax=255,aspect = "equal")
#axEye.plot(eye_params.sel(ellipse_params = 'X0')[fr]/2,eye_params.sel(ellipse_params = 'Y0')[fr]/2,'r.')
axEye.set_xlim(40,200); axEye.set_ylim(0,120)

axWorld.cla();  axWorld.axis('off'); 
axWorld.imshow(world_vid[worldFr,:,:],'gray',vmin=0,vmax=255,aspect = "equal")

axTop.axis('off')
axTop.imshow(top_vid[0,:,:],'gray',vmin=0,vmax=255,aspect = 'equal')
   
#plot eye position
axTheta.cla()
axTheta.plot(eyeT,0.5*eye_params.sel(ellipse_params = 'theta')*180/3.14159)
axTheta.set_xlim(tr[0],tr[1]); 
axTheta.set_ylabel('theta - deg'); axTheta.set_ylim(-20,20)

# grab bag plot -speed / eye vel

#speed
axdTheta.plot(topT[0:-1],spd); axdTheta.set_ylabel('speed cm/sec')
axdTheta.set_ylim([0, 25])

#eve velocity
#axdTheta.plot(eyeT[0:-1],dEye*60/2); ax.set_ylabel('dtheta')
#sacc = np.transpose(np.where(np.abs(dEye)>10))
#axdTheta.plot(sacc,np.sign(dEye[sacc])*20,'.')
#axdTheta.set_ylim(-10*60,10*60); axdTheta.set_ylabel('eye vel - deg/sec')

axdTheta.set_xlim(tr[0],tr[1]); 

#plot contrast or radius
#axVid.plot(worldT,contrast/2.5)
#axVid.set_ylim([0, 1.1]); axVid.set_ylabel('contrast')

axVid.plot(eyeT,0.5*(eye_params.sel(ellipse_params = 'longaxis') +eye_params.sel(ellipse_params = 'shortaxis')) )
axVid.set_xlabel('radius'); axVid.set_ylim([0, 50])
axVid.set_xlim(tr[0],tr[1]);



# plot spikes
axR.fontsize = 20
for i,ind in enumerate(goodcells.index):
    axR.vlines(goodcells.at[ind,'spikeT'],i-0.25,i+0.25,'k',linewidth=0.25)
axR.set_xlim(tr[0],tr[1]); axR.set_xlabel('secs'); axR.set_ylabel('unit #')
#axR.spines['right'].set_visible(False)
#axR.spines['top'].set_visible(False)

plt.tight_layout()

plt.savefig('cams_eyes_R_move_spikes_092820_wn_0_30.pdf')

In [ ]:
# set up interpolators for eye and world videos
eyeInterp = interp1d(eyeT,eye_vid,axis=0)
worldInterp = interp1d(worldT,world_vid,axis=0)

In [ ]:
# make movie and sound

#unit to plot/record
this_unit = 1

#set up figure
fig = plt.figure(figsize = (6,8))
gs = fig.add_gridspec(4,2)
axEye = fig.add_subplot(gs[0,0])
axWorld = fig.add_subplot(gs[0,1])
axTheta = fig.add_subplot(gs[1,:])
axdTheta = fig.add_subplot(gs[2,:])
axR = fig.add_subplot(gs[3,:])
#axRad = fig.add_subplot(gs[3,:])

#timerange and center frame (only)
tr = [0, 30]
fr = np.mean(tr) # time for frame
eyeFr = np.abs(eyeT-fr).argmin(dim = "frame")
worldFr = np.abs(worldT-fr).argmin(dim = "frame")

axEye.cla(); axEye.axis('off'); 
axEye.imshow(eye_vid[eyeFr,:,:],'gray',vmin=0,vmax=255,aspect = "equal")
#axEye.plot(eye_params.sel(ellipse_params = 'X0')[fr]/2,eye_params.sel(ellipse_params = 'Y0')[fr]/2,'r.')
axEye.set_xlim(40,200); axEye.set_ylim(0,120)

axWorld.cla();  axWorld.axis('off'); 
axWorld.imshow(world_vid[worldFr,:,:],'gray',vmin=0,vmax=255,aspect = "equal")
   
#plot eye position
axTheta.cla()
axTheta.plot(eyeT,eye_params.sel(ellipse_params = 'theta')*180/3.14159)
axTheta.set_xlim(tr[0],tr[1]); 
axTheta.set_ylabel('theta - deg'); axTheta.set_ylim(-30,30)

# plot eye velocity
axdTheta.cla()
#axdTheta.plot(eyeT[0:-1],dEye*60); ax.set_ylabel('dtheta')
#sacc = np.transpose(np.where(np.abs(dEye)>10))
#axdTheta.plot(sacc,np.sign(dEye[sacc])*20,'.')
#axdTheta.set_xlim(tr[0],tr[1]); 
#axdTheta.set_ylim(-23*60,30*60); axdTheta.set_ylabel('eye vel - deg/sec')

# plot speed
axdTheta.plot(topT[0:-1],spd); axdTheta.set_ylabel('speed cm/sec')
axdTheta.set_ylim([0, 25])
axdTheta.set_xlim(tr[0],tr[1]); 

#plot radius?
#axRad.cla()
#axRad.plot(eye_params.sel(ellipse_params = 'longaxis')[frameRange])
#axRad.set_xlim(0,frameRange[-1]-frameRange[0]); 
#axRad.set_ylabel('radius'); axRad.set_xlabel('frame #'); axRad.set_ylim(0,40)

# plot spikes
axR.fontsize = 20
for i,ind in enumerate(goodcells.index):
    axR.vlines(goodcells.at[ind,'spikeT'],i-0.25,i+0.25,'k',linewidth=0.5)
axR.vlines(goodcells.at[units[this_unit],'spikeT'],this_unit-0.25,this_unit+0.25,'b',linewidth=0.5)

axR.set_xlim(tr[0],tr[1]); axR.set_ylim(-0.5 , 4); axR.set_xlabel('secs'); axR.set_ylabel('unit #')
axR.spines['right'].set_visible(False)
axR.spines['top'].set_visible(False)

plt.tight_layout()

# now animate
writer = FFMpegWriter(fps=30)
with writer.saving(fig, "eye_world_spikes_092820_wn1_1.mp4", 100):
    for t in np.arange(tr[0],tr[1],1/30):
        
        # show eye and world frames
        axEye.cla(); axEye.axis('off'); 
        axEye.imshow(eyeInterp(t),'gray',vmin=0,vmax=255,aspect = "equal")
        axEye.set_xlim(0,160); axEye.set_ylim(0,120)
        
        axWorld.cla(); axWorld.axis('off'); 
        axWorld.imshow(np.flipud(worldInterp(t)),'gray',vmin=0,vmax=255,aspect = "equal")
        
        #plot line for time, then remove
        ln = axR.vlines(t,-0.5,30,'b')
        writer.grab_frame()
        ln.remove()
        
###



# generate wave file
sp =np.array(ephys_data.at[units[this_unit],'spikeT'])-tr[0]
sp = sp[sp>0]
datarate = 30000
f = 440.0       # sound frequency (Hz)
# Compute waveform samples
tmax = tr[1]-tr[0]
t = np.linspace(0, tr[1]-tr[0], (tr[1]-tr[0])*datarate,endpoint=False)
x = np.zeros(np.size(t))
for spt in sp[sp<tmax]:
    x[np.int64(spt*datarate) : np.int64(spt*datarate +30)] = 1
    x[np.int64(spt*datarate)+31 : np.int64(spt*datarate +60)] =- 1
plt.plot(x)
    

# Write the samples to a file
wavio.write("wn_spike1.wav", x, datarate, sampwidth=1)

In [ ]:
#https://cloudconvert.com/wav-to-mp3
# https://www.kapwing.com/tools/add-music-to-video

In [ ]:
# normalize world video
im_std = np.std(world_vid,axis=0)
im_std = 32;
img_norm = (world_vid-np.mean(world_vid,axis=0))/im_std
#plt.imshow(im_std)
#plt.colorbar()

In [ ]:
plt.imshow(img_norm[0,:,:],vmin=-2,vmax=2)

In [ ]:
#calculate image contrast
contrast = np.empty(worldT.size)
for i in range(worldT.size):
    contrast[i] = np.std(img_norm[i,:,:])
plt.plot(worldT[0:1000],contrast[0:1000])
plt.xlabel('time')
plt.ylabel('contrast')


In [ ]:
#set up timebase for subsequent analysis
dt = 0.1
t = np.arange(0, np.max(worldT),dt)

In [ ]:
# interpolate and plot contrast
newc =interp1d(worldT,contrast)
contrast_interp = newc(t[0:-1])
contrast_interp.shape
plt.plot(contrast_interp[0:600])

In [ ]:
# calculate firing rate at new timebase
ephys_data['rate'] = nan
ephys_data['rate'] = ephys_data['rate'].astype(object)
for i,ind in enumerate(ephys_data.index):
    ephys_data.at[ind,'rate'],bins = np.histogram(ephys_data.at[ind,'spikeT'],t                                             )
ephys_data['rate']= ephys_data['rate']/dt
goodcells = ephys_data.loc[ephys_data['group']=='good']
goodcells

In [ ]:
# plot firing rates
for i, ind in enumerate(goodcells.index):
    plt.subplot(7,2,i+1)
    plt.plot(t[0:-1],goodcells.at[ind,'rate'])

In [ ]:
#scatter of contrast vs rate
plt.figure(figsize = (12,4))
for i, ind in enumerate(goodcells.index):
    plt.subplot(7,2,i+1)
    plt.plot(contrast_interp,goodcells.at[ind,'rate'],'.')
plt.xlabel('contrast')
plt.ylabel('rate')

In [ ]:
# calculate contrast - response functions
# mean firing rate in timebins correponding to contrast ranges
resp = np.empty((6,20))
crange = np.arange(0,2,0.1)
for i,ind in enumerate(goodcells.index):
    for c,cont in enumerate(crange):
        resp[i,c] = np.mean(goodcells.at[ind,'rate'][(contrast_interp>cont) & (contrast_interp<(cont+0.1))])
plt.plot(crange,np.transpose(resp))
#plt.ylim(0,10)
plt.xlabel('contrast')
plt.ylabel('sp/sec')
    

In [ ]:
# plot individual contrast response functions in subplots
fig = plt.figure(figsize = (12,8))
for i, ind in enumerate(goodcells.index):
    plt.subplot(3,5,i+1)
    plt.plot(crange[2:-3],resp[i,2:-3])
    plt.ylim([0 , max(resp[i,3:-3])*1.2])
    plt.xlabel('contrast a.u.'); plt.ylabel('sp/sec')
plt.tight_layout()
plt.savefig('contrast_resp_101520_fm1_merge.pdf')

In [ ]:
# create interpolator for movie data so we can evaluate at same timebins are firing rate
movInterp = interp1d(worldT,img_norm,axis=0)

In [ ]:
# calculate spike-triggered average
sta = 0
lag = -0.1
plt.figure(figsize = (12,8))
for c, ind in enumerate(goodcells.index):
    sta =0
    totalR = 0;
    r = goodcells.at[ind,'rate']
    for i in range(np.uint(t.size*0)+1000,np.uint(t.size*1)-1000):
        sta = sta+r[i]*(movInterp(t[i]-lag))
        totalR = totalR + r[i]
    plt.subplot(3,5,c+1)
    sta = sta/totalR
    #sta[abs(sta)<0.1]=0
    plt.imshow((sta - np.mean(sta)),vmin=-0.5,vmax=0.5,cmap = 'jet')
    plt.title(totalR*dt)
plt.tight_layout()
plt.savefig('sta_101520_vm045_fm_merge.pdf')


In [ ]:
# calculate spike-triggered average
spike_corr = 1 + 0.125/1200
sta = 0
lag = 0.075
lagRange = np.arange(0,0.25,0.05)
plt.figure(figsize = (12,8))
for c, ind in enumerate(goodcells.index):
    sp = goodcells.at[ind,'spikeT'].copy()
    for  lagInd, lag in enumerate(lagRange):
        sta = 0; nsp = 0
        for s in sp:
            if (s-lag >5) & ((s-lag)*spike_corr <np.max(worldT)):
                nsp = nsp+1
                fr = movInterp((s-lag)*spike_corr)
                sta = sta+ fr
        plt.subplot(6,6,(c*6)+lagInd + 1)
        sta = sta/nsp
    #sta[abs(sta)<0.1]=0
        plt.imshow((sta-np.mean(sta) ),vmin=-0.35,vmax=0.35,cmap = 'jet')
plt.tight_layout()
plt.savefig(name_base + '_sta.pdf')

In [ ]:
# calculate spike-triggered variance
sta = 0
lag = 0.075
plt.figure(figsize = (12,8))
for c, ind in enumerate(goodcells.index):
    sta = 0
    r = goodcells.at[ind,'rate']
    for i in range(5,t.size-10):
        sta = sta+r[i]*(movInterp(t[i]-lag))**2
    plt.subplot(3,5,c+1)
    sta = sta/np.sum(r)
    plt.imshow(sta)
plt.tight_layout()

In [ ]:
# calculate saccade-locked psth
fig = plt.figure(figsize = (12,8))
trange = np.arange(-5,5.1,0.1)
sthresh = 5;
upsacc = eyeT[np.append(dEye,0)>sthresh].copy()
upsacc = upsacc[upsacc>5]  # buffer at beginning
downsacc= eyeT[np.append(dEye,0)<-sthresh].copy()
downsacc = downsacc[downsacc>5] #buffer at beginning
upsacc_avg = np.zeros((units.size,trange.size))
downsacc_avg = np.zeros((units.size,trange.size))
for i, ind in enumerate(goodcells.index):
    rateInterp = interp1d(t[0:-1],goodcells.at[ind,'rate'])
    for s in upsacc:
        upsacc_avg[i,:] = upsacc_avg[i,:]+ rateInterp(np.array(s)+trange)/upsacc.size
    for s in downsacc:
        downsacc_avg[i,:]= downsacc_avg[i,:]+ rateInterp(np.array(s)+trange)/upsacc.size
    plt.subplot(3,5,i+1)
    plt.plot(trange,upsacc_avg[i,:])
    plt.plot(trange,downsacc_avg[i,:],'r')
    plt.vlines(0,0,np.max(upsacc_avg[i,:]*0.2),'r')
    plt.ylim([0, np.max(np.concatenate((upsacc_avg[i,:],downsacc_avg[i,:]),axis = None))*1.2])
    plt.ylabel('sp/sec')
plt.tight_layout()
plt.savefig('saccade_101520_fm1_merge.pdf')
        


In [ ]:
fig = plt.figure(figsize = (12,8))
ns = np.zeros((6,len(upsacc)+1))
for i, ind in enumerate(goodcells.index):
    sp = np.array(goodcells.at[units[i],'spikeT'])*spike_corr
    plt.subplot(2,3,i+1)
    n = 0
    for s in upsacc:
        n= n+1
        sd = np.abs(sp-np.array(s))<1
        sacc_sp = sp[sd]
        ns[i,n] =len(sacc_sp)
        plt.vlines(sacc_sp-np.array(s),n-0.25,n+0.25)
    plt.xlim(-1,1)
   # plt.ylim(0,25)
plt.savefig('saccade_upraster_101520_fm1_merge.pdf')

In [ ]:
np.max(t)

In [ ]:
fig = plt.figure(figsize = (12,8))
for i, ind in enumerate(goodcells.index):
    sp = np.array(goodcells.at[units[i],'spikeT'])*spike_corr
    plt.subplot(2,3,i+1)
    n = 0
    for s in downsacc:
        n= n+1
        sd = np.abs(sp-np.array(s))<10
        sacc_sp = sp[sd]
        plt.vlines(sacc_sp-np.array(s),n-0.25,n+0.25)
    plt.xlim(-1,1)

In [ ]:
n_units = len(goodcells)
sp_range = np.arange(0,25,2)
spT = topT[0:-1].copy()
useSpT = spT[spT<t[-2]].copy()
useSpd = spd[spT<t[-2]].copy()
sp_scatter = np.zeros((n_units,len(useSpd)))
sp_tuning = np.zeros((n_units,len(sp_range)-1))
sp_tuning_err = sp_tuning.copy()
for i, ind in enumerate(goodcells.index):
    rateInterp = interp1d(t[0:-1],goodcells.at[ind,'rate'])
    sp_scatter[i,:] = rateInterp(useSpT)
    plt.subplot(2,3,i+1)
    plt.scatter(useSpd,sp_scatter[i,:])
    plt.xlim([0, 50])
    for j in range(len(sp_range)-1):
        usePts =(useSpd>sp_range[j]) & (useSpd<sp_range[j+1])
        sp_tuning[i,j] = np.mean(sp_scatter[i,usePts])
        sp_tuning_err[i,j] = np.std(sp_scatter[i,usePts])/np.sqrt(np.count_nonzero(usePts))

In [ ]:
plt.hist2d(useSpd,sp_scatter[0,:].T,range = [[0,10],[0,50]])

In [ ]:
usePts =(useSpd>sp_range[j]) & (useSpd<sp_range[j+1])
sp_tuning[i,j] = np.mean(sp_scatter[i,usePts])
sp_tuning_err[i,j] = np.std(sp_scatter[i,usePts])/np.sqrt(np.count_nonzero(usePts))

In [ ]:
sp_scatter[i,usePts]

In [ ]:
for i in range(n_units):
    plt.subplot(2,3,i+1)
    plt.errorbar(sp_range[:-1],sp_tuning[i,:],yerr=sp_tuning_err[i,:])
    plt.ylim(0,15)

In [ ]:
plt.plot(useSpT,sp_scatter[0,:])
plt.plot(useSpT, useSpd*5)
plt.xlim([300, 325])
plt.ylim([0, 100])

In [ ]:
dt

In [ ]:
plt.plot(spd)
plt.ylim(0,20)

In [ ]:
Rnorm = (eyeR - np.mean(eyeR))/np.std(eyeR)
plt.plot(eyeT,Rnorm)
plt.xlim([0,60])


In [ ]:
# plot rate vs pupil
n_units = len(goodcells)
R_range = np.arange(-4,4,0.5)
useEyeT = eyeT[(eyeT<t[-2]) & (eyeT>t[0])].copy()
useR = Rnorm[(eyeT<t[-2]) & (eyeT>t[0])].copy()
R_scatter = np.zeros((n_units,len(useR)))
R_tuning = np.zeros((n_units,len(R_range)-1))
R_tuning_err =R_tuning.copy()
for i, ind in enumerate(goodcells.index):
    rateInterp = interp1d(t[0:-1],goodcells.at[ind,'rate'])
    R_scatter[i,:] = rateInterp(useEyeT)
    for j in range(len(R_range)-1):
        usePts =(useR>R_range[j]) & (useR<R_range[j+1])
        R_tuning[i,j] = np.mean(R_scatter[i,usePts])
        R_tuning_err[i,j] = np.std(R_scatter[i,usePts])/np.sqrt(np.count_nonzero(usePts))

fig = plt.figure(figsize = (3*np.ceil(n_units/2),6))
for i in range(n_units):
    plt.subplot(2,np.ceil(n_units/2),i+1)
    plt.errorbar(R_range[:-1],R_tuning[i,:],yerr=R_tuning_err[i,:])
    plt.ylim(0,np.nanmax(R_tuning[i,:]*1.2))
    plt.xlim([-2, 3])
    plt.xlabel('normalized pupil R'); plt. ylabel('sp/sec'); plt.title(i)
        
        
plt.tight_layout()

plt.savefig(name_base + '_pupil_R.pdf')

In [ ]:
eyeTheta = eye_params.sel(ellipse_params = 'theta').copy()
thetaNorm = (eyeTheta - np.mean(eyeTheta))/np.std(eyeTheta)
plt.plot(eyeT[0:3600],thetaNorm[0:3600])
plt.xlabel('secs'); plt.ylabel('normalized eye theta')

In [ ]:
# plot rate vs pupil
n_units = len(goodcells)
th_range = np.arange(-2,3,0.5)
useEyeT = eyeT[(eyeT<t[-2]) & (eyeT>t[0])].copy()
useTh = thetaNorm[(eyeT<t[-2]) & (eyeT>t[0])].copy()
th_scatter = np.zeros((n_units,len(useR)))
th_tuning = np.zeros((n_units,len(th_range)-1))
th_tuning_err =th_tuning.copy()
for i, ind in enumerate(goodcells.index):
    rateInterp = interp1d(t[0:-1],goodcells.at[ind,'rate'])
    th_scatter[i,:] = rateInterp(useEyeT)
    for j in range(len(th_range)-1):
        usePts =(useTh>th_range[j]) & (useTh<th_range[j+1])
        th_tuning[i,j] = np.mean(th_scatter[i,usePts])
        th_tuning_err[i,j] = np.std(th_scatter[i,usePts])/np.sqrt(np.count_nonzero(usePts))

fig = plt.figure(figsize = (3*np.ceil(n_units/2),6))
for i in range(n_units):
    plt.subplot(2,np.ceil(n_units/2),i+1)
    plt.errorbar(th_range[:-1],th_tuning[i,:],yerr=th_tuning_err[i,:])
    plt.ylim(0,np.nanmax(th_tuning[i,:]*1.2))
    plt.xlim([-2, 2])
    plt.xlabel('normalized pupil theta'); plt. ylabel('sp/sec'); plt.title(i)
        
        
plt.tight_layout()
plt.savefig(name_base + '_pupil_theta.pdf')

In [ ]:
np.shape(img_norm)[0]

In [ ]:
nf =np.shape(img_norm)[0]
cc = np.zeros(nf-1)
imgz = np.zeros(np.shape(img_norm))
for f in range(nf):
    imgz[f,:,:] = (img_norm[f,:,:] - np.mean(img_norm[f,:,:]))/np.std(img_norm[f,:,:])  
for f in range(nf-1):
    cc[f]= np.mean(imgz[f,:,:]*imgz[f+1,:,:])

In [ ]:
eyeTh = np.array(eye_params.sel(ellipse_params = 'theta')*180/3.1415)
eyePhi = np.array(eye_params.sel(ellipse_params = 'phi')*180/3.1415)
eyeTh = eyeTh-np.nanmean(eyeTh)
eyePhi = eyePhi - np.nanmean(eyePhi)

thInterp= interp1d(eyeT,eyeTh,bounds_error = False, fill_value = np.nan)
phInterp = interp1d(eyeT,eyePhi,bounds_error = False, fill_value = np.nan)


th_cal = 0;
phi_cal = -0.1;
sx = thInterp(worldT)*th_cal
sy = phInterp(worldT)*phi_cal
movshift = np.zeros(np.shape(img_norm))
for f in range(len(sx)):
    movshift[f,:,:] = imshift(img_norm[f,:,:],(sy[f],sx[f]),mode = 'constant',cval = np.nan)
    
    

In [ ]:
nf =np.shape(img_norm)[0]
cc = np.zeros(nf-1)
imgz = np.zeros(np.shape(img_norm))
for f in range(nf):
    imgz[f,:,:] = (movshift[f,:,:] - np.nanmean(movshift[f,:,:]))/np.nanstd(movshift[f,:,:])  
for f in range(nf-1):
    cc[f]= np.nanmean(imgz[f,:,:]*imgz[f+1,:,:])
plt.plot(cc[0:6000])
plt.ylim([0.8,1])
print(np.nanmean(cc))
print(np.nanmedian(cc))

In [ ]:
for f in range(8):
    plt.subplot(2,4,f+1)
    plt.imshow(np.flipud(movshift[f,:,:]))
    plt.title("th{th: 0.1f} phi {phi:0.1f}".format(th = sx[f],phi=sy[f]))

In [ ]:
np.array(eye_params.sel(ellipse_params = 'theta'))

In [ ]:
f0 = 941
plt.figure(figsize =(12,4))
for f in range(-2,3):
    plt.subplot(1,5,f+3)
    plt.imshow(np.flipud(img_norm[f+f0,:,:]))

In [ ]:
np.where(cc>0.99)